# 주간 차트 데이터 EDA

### 1. 곡별 집계
1위=100점 ... 100위=1점 으로 계산  
기간별 곡의 누적점수 집계

In [6]:
## 주간 차트 데이터 EDA

import pandas as pd

# 1. 차트 데이터 불러오기
df = pd.read_csv("mma_2025_period.csv")

# 2. 순위를 숫자형으로 변환 및 점수 계산 (1위 100점 ~ 100위 1점)
df['순위'] = pd.to_numeric(df['순위'])
df['점수'] = 101 - df['순위']

# 3. 곡별 집계
# 누적점수, 1위 횟수, TOP 10(1~10위) 횟수, 전체 차트인 주수를 계산합니다.
analysis = df.groupby(['곡명', '아티스트']).agg(
    누적점수=('점수', 'sum'),
    일위횟수=('순위', lambda x: (x == 1).sum()),
    TOP10횟수=('순위', lambda x: (x <= 10).sum()),
    총차트인주수=('순위', 'count')
).reset_index()

analysis['누적점수'] = analysis['누적점수'].map(int)

# 4. 결과 정렬 (누적 점수 기준)
analysis = analysis.sort_values(by='누적점수', ascending=False)

# 5. 상위 20개 출력
print("2025 심사기간 차트 성적 TOP 20:")
print(analysis.head(20).to_string(index=False))

# 6. 결과 저장
# analysis.to_csv("chart_pure_analysis_2025.csv", index=False, encoding="utf-8-sig")

2025 심사기간 차트 성적 TOP 20:
                                        곡명                    아티스트  누적점수  일위횟수  TOP10횟수  총차트인주수
                                  Drowning                   WOODZ  5139     2       46      55
                                  Whiplash                   aespa  5102     0       41      55
                                    나는 반딧불                     황가람  4885     0       31      55
                                      APT.               로제 (ROSÉ)  4836     4       26      55
            HOME SWEET HOME (feat. 태양, 대성)                G-DRAGON  4796    10       28      52
                                     HAPPY             DAY6 (데이식스)  4777     0       16      55
                          내게 사랑이 뭐냐고 물어본다면                     로이킴  4492     0        7      55
                                       천상연                     이창섭  4377     0        0      55
                                       소나기          이클립스 (ECLIPSE)  4374     0        0      55
                

### 2. 아티스트 통합 점수

아티스트의 모든 노래 통합 점수

In [10]:
artist_score = analysis.groupby('아티스트')['누적점수'].sum().sort_values(ascending=False)
artist_score.head(20)

아티스트
DAY6 (데이식스)       20405
aespa             17585
G-DRAGON          15771
IVE (아이브)         11325
이무진               10380
로제 (ROSÉ)          9837
QWER               8339
NewJeans           8150
아이유                7829
황가람                6905
임영웅                6900
제니 (JENNIE)        5577
BOYNEXTDOOR        5551
이창섭                5370
WOODZ              5149
아일릿(ILLIT)         4844
로이킴                4839
우디 (Woody)         4428
이클립스 (ECLIPSE)     4374
PLAVE              4138
Name: 누적점수, dtype: int64

### 3. 점수효율성
점수 효율성 = 누적 점수 ÷ 총 차트인 곡 수

In [15]:
# 아티스트별 곡 수 계산
artist_song_count = analysis.groupby('아티스트')['곡명'].count()

# artist_song_count.sort_values(ascending=False)
# # 효율성 계산 (점수 / 곡 수)
efficiency = (artist_score / artist_song_count).sort_values(ascending=False)

print("아티스트별 곡당 평균 화력(효율성) TOP 10:")
print(efficiency.head(10))

아티스트별 곡당 평균 화력(효율성) TOP 10:
아티스트
이클립스 (ECLIPSE)             4374.0
조째즈                        3790.0
너드커넥션 (Nerd Connection)    3614.0
순순희(지환)                    3463.0
황가람                        3452.5
로제 (ROSÉ)                  3279.0
BABYMONSTER                3076.0
10CM                       3069.0
잔나비                        2979.0
임재현                        2915.0
dtype: float64


In [37]:
## 주간 차트 데이터 EDA

import pandas as pd

# 1. 차트 데이터 불러오기
df = pd.read_csv("mma_2025_period.csv")
df.head()

,연도,주차,순위,곡명,아티스트,앨범,좋아요,이미지URL,마감일_문자열,마감일
0,2024,10.28~11.03,1.0,APT.,로제 (ROSÉ),APT.,219309,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03
1,2024,10.28~11.03,2.0,Whiplash,aespa,Whiplash - The 5th Mini Album,150132,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03
2,2024,10.28~11.03,3.0,UP (KARINA Solo),aespa,SYNK : PARALLEL LINE - Special Digital Single,117304,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03
3,2024,10.28~11.03,4.0,Mantra,제니 (JENNIE),Mantra,79619,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03
4,2024,10.28~11.03,5.0,HAPPY,DAY6 (데이식스),Fourever,208315,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03


In [38]:
df['시작일'] = df['연도'].astype(str) + '.' + df['주차'].str.split('~').str[0].str.strip()
df.head()

,연도,주차,순위,곡명,아티스트,앨범,좋아요,이미지URL,마감일_문자열,마감일,시작일
0,2024,10.28~11.03,1.0,APT.,로제 (ROSÉ),APT.,219309,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03,2024.10.28
1,2024,10.28~11.03,2.0,Whiplash,aespa,Whiplash - The 5th Mini Album,150132,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03,2024.10.28
2,2024,10.28~11.03,3.0,UP (KARINA Solo),aespa,SYNK : PARALLEL LINE - Special Digital Single,117304,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03,2024.10.28
3,2024,10.28~11.03,4.0,Mantra,제니 (JENNIE),Mantra,79619,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03,2024.10.28
4,2024,10.28~11.03,5.0,HAPPY,DAY6 (데이식스),Fourever,208315,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024-11-03,2024.10.28


In [39]:
df['마감일'] = df['마감일_문자열']
df.drop(columns='마감일_문자열', inplace=True)
df.head()

,연도,주차,순위,곡명,아티스트,앨범,좋아요,이미지URL,마감일,시작일
0,2024,10.28~11.03,1.0,APT.,로제 (ROSÉ),APT.,219309,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024.10.28
1,2024,10.28~11.03,2.0,Whiplash,aespa,Whiplash - The 5th Mini Album,150132,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024.10.28
2,2024,10.28~11.03,3.0,UP (KARINA Solo),aespa,SYNK : PARALLEL LINE - Special Digital Single,117304,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024.10.28
3,2024,10.28~11.03,4.0,Mantra,제니 (JENNIE),Mantra,79619,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024.10.28
4,2024,10.28~11.03,5.0,HAPPY,DAY6 (데이식스),Fourever,208315,https://cdnimg.melon.co.kr/cm2/album/images/11...,2024.11.03,2024.10.28


In [40]:
df.to_csv('mma_2025.csv',index=False, encoding="utf-8-sig")